The torch.nn namespace provides all the building blocks you need to build your own neural network. Every module in PyTorch subclasses the nn.Module.

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

Get Device for Training

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cpu device


Define the Class:<br/>
We define our neural network by subclassing nn.Module, and initialize the neural network layers in __init__.Every nn.Module subclass implements the operations on input data in the forward method.

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

We create an instance of NeuralNetwork, and move it to the device, and print its structure

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


To use the model, we pass it the input data. This executes the model’s forward, along with some background operations. Do not call model.forward() directly.<br/>
Calling the model on the input returns a 10-dimentional tensor with raw predicted values for each class. We get the prediction probabilities by passing it through an instance of the nn.Softmax module.

In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Prediction class: {y_pred}")

Prediction class: tensor([7])


Model Layers:<br/>
Lets break down the layers in the FashionMNIST model. To illlustrate it, we will take a sample minibatch of 3 images of size 28*28 and see what happens to it as we pass it through the network.

In [6]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


We initialize the nn.Flatten layer to convert each 2D 28*28 image into a contiguous array of 784 pixel values(the minibatch dimention(at dim=0) is maintained)

In [7]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


The linear layer is a module that applies a linear transformation on the input using its stored weights and biases.

In [8]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


nn.ReLU

In [9]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-2.8424e-01, -3.6564e-01, -2.5302e-01,  5.8876e-02,  4.8362e-01,
         -1.7341e-01,  1.0275e-01, -1.1780e-01,  3.3319e-01, -3.5459e-01,
         -1.4355e-01,  1.2483e-02, -6.3081e-01,  1.1364e-01, -3.5878e-01,
         -4.3281e-01,  5.0308e-01,  2.1634e-01,  5.1774e-03, -2.2465e-01],
        [ 1.8880e-02, -4.7880e-01, -1.5432e-01,  6.2890e-01,  3.6866e-01,
          1.3517e-01, -6.2058e-02, -1.1721e-02,  1.6206e-01, -2.3175e-01,
         -3.7467e-02, -2.1244e-02, -4.2617e-01,  1.9669e-01, -1.6442e-01,
         -2.5863e-01,  4.4242e-01, -3.4387e-01, -6.5438e-02,  5.1222e-02],
        [-6.0355e-02, -1.9922e-01,  2.7759e-02,  9.8445e-02,  5.8497e-01,
          4.4959e-02,  8.6361e-02,  2.2300e-01,  1.6484e-01, -4.9559e-01,
         -1.0828e-01,  7.6962e-02, -5.4979e-01,  2.5035e-01, -6.3112e-01,
         -1.1005e-04,  3.6009e-01, -2.1212e-01,  3.0863e-01, -2.5120e-01]],
       grad_fn=<AddmmBackward>)


After ReLU: tensor([[0.0000, 0.0000, 0.0000, 0.0589, 0.4836, 

nn.Sequential is an ordered container of modules.

In [10]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

The last linear layer of the neural network returns logits - raw values in [-infty, infty] - which are passed to the nn.Softmax module. The logits are scaled to values [0, 1] representing the model’s predicted probabilities for each class.<br/>
dim parameter indicates the dimension along which the values must sum to 1.